## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [12]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [13]:
train, test = keras.datasets.cifar10.load_data()

In [14]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [15]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [16]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [17]:
LEARNING_RATE = 1e-3
EPOCHS = 10 # IF you feel too run to finish, try to make it smaller
BATCH_SIZE = 1024 

In [20]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
reduce_lr_factor = [0.9, 0.6, 0.3] # don't receive factor >=1
redice_lr_patient = [15, 10, 5]


In [ ]:
import itertools
results = {}
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i" % (i, reduce_factor, reduce_patient))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    
    """Code Here
    設定 reduce learning rate 的 callback function
    """
    reduce_lr = ReduceLROnPlateau(factor=reduce_factor, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=reduce_patient, 
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, reduce_factor: 0.90, reduce_patient: 15
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_22 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_23 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalizati

50000/50000 [==============================] - 9s 182us/step - loss: 1.3992 - acc: 0.5120 - val_loss: 1.5388 - val_acc: 0.4570
Epoch 8/10
50000/50000 [==============================] - 10s 203us/step - loss: 1.3655 - acc: 0.5240 - val_loss: 1.5228 - val_acc: 0.4611
Epoch 9/10
50000/50000 [==============================] - 10s 191us/step - loss: 1.3316 - acc: 0.5346 - val_loss: 1.5087 - val_acc: 0.4654
Epoch 10/10
50000/50000 [==============================] - 10s 203us/step - loss: 1.3030 - acc: 0.5462 - val_loss: 1.4912 - val_acc: 0.4707
Numbers of exp: 3, reduce_factor: 0.60, reduce_patient: 15
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________


50000/50000 [==============================] - 9s 188us/step - loss: 1.6196 - acc: 0.4383 - val_loss: 1.6940 - val_acc: 0.4132
Epoch 4/10
50000/50000 [==============================] - 10s 203us/step - loss: 1.5456 - acc: 0.4616 - val_loss: 1.6325 - val_acc: 0.4297
Epoch 5/10
50000/50000 [==============================] - 10s 190us/step - loss: 1.4871 - acc: 0.4825 - val_loss: 1.5928 - val_acc: 0.4384
Epoch 6/10
50000/50000 [==============================] - 9s 190us/step - loss: 1.4446 - acc: 0.4965 - val_loss: 1.5659 - val_acc: 0.4548
Epoch 7/10
50000/50000 [==============================] - 10s 191us/step - loss: 1.4057 - acc: 0.5092 - val_loss: 1.5488 - val_acc: 0.4534
Epoch 8/10
50000/50000 [==============================] - 9s 187us/step - loss: 1.3672 - acc: 0.5238 - val_loss: 1.5200 - val_acc: 0.4705
Epoch 9/10
50000/50000 [==============================] - 10s 192us/step - loss: 1.3357 - acc: 0.5368 - val_loss: 1.5223 - val_acc: 0.4691
Epoch 10/10
50000/50000 [================

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 12s 235us/step - loss: 2.1800 - acc: 0.2800 - val_loss: 2.0935 - val_acc: 0.3255
Epoch 2/10
50000/50000 [==============================] - 10s 197us/step - loss: 1.7272 - acc: 0.4004 - val_loss: 1.8054 - val_acc: 0.3894
Epoch 3/10
50000/50000 [==============================] - 10s 192us/step - loss: 1.5999 - acc: 0.4399 - val_loss: 1.6808 - val_acc: 0.4192
Epoch 4/10
50000/50000 [==============================] - 10s 192us/step - loss: 1.5286 - acc: 0.4669 - val_loss: 1.6661 - val_acc: 0.4144
Epoch 5/10
50000/50000 [==============================] - 10s 205us/step - loss: 1.4732 - acc: 0.4854 - val_loss: 1.5978 - val_acc: 0.4398
Epoch 6/10
50000/50000 [==============================] - 11s 224us/step - loss: 1.4271 - acc: 0.5040 - val_loss: 1.5958 - val_acc: 0.4435
Epoch 7/10
50000/50000 [==============================] - 10s 205us/step - loss: 1.3855 - acc: 0.5189 - val_loss: 1.

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 12s 247us/step - loss: 1.8116 - acc: 0.3624 - val_loss: 1.7020 - val_acc: 0.4030
Epoch 2/10
50000/50000 [==============================] - 10s 207us/step - loss: 1.5694 - acc: 0.4407 - val_loss: 1.6406 - val_acc: 0.4133
Epoch 3/10
50000/50000 [==============================] - 10s 202us/step - loss: 1.4811 - acc: 0.4730 - val_loss: 1.7703 - val_acc: 0.3803
Epoch 4/10
50000/50000 [==============================] - 10s 208us/step - loss: 1.4047 - acc: 0.5024 - val_loss: 1.7538 - val_acc: 0.3805
Epoch 5/10
50000/50000 [==============================] - 11s 212us/step - loss: 1.3489 - acc: 0.5215 - val_loss: 1.5788 - val_acc: 0.4407
Epoch 6/10
50000/50000 [==============================] - 11s 226us/step - loss: 1.2954 - acc: 0.5391 - val_loss: 1.6236 - val_acc: 0.4217
Epoch 7/10
50000/50000 [==============================] - 11s 219us/step - loss: 1.2511 - acc: 0.5550 - val_loss: 1.

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 14s 287us/step - loss: 1.8054 - acc: 0.3665 - val_loss: 1.7670 - val_acc: 0.3845
Epoch 2/10
50000/50000 [==============================] - 10s 209us/step - loss: 1.5631 - acc: 0.4443 - val_loss: 1.6325 - val_acc: 0.4227
Epoch 3/10
50000/50000 [==============================] - 10s 194us/step - loss: 1.4668 - acc: 0.4791 - val_loss: 1.6610 - val_acc: 0.4163
Epoch 4/10
50000/50000 [==============================] - 12s 233us/step - loss: 1.3995 - acc: 0.5010 - val_loss: 1.5850 - val_acc: 0.4269
Epoch 5/10
50000/50000 [==============================] - 10s 199us/step - loss: 1.3499 - acc: 0.5218 - val_loss: 1.5949 - val_acc: 0.4371
Epoch 6/10
50000/50000 [==============================] - 10s 205us/step - loss: 1.2997 - acc: 0.5381 - val_loss: 1.6311 - val_acc: 0.4339
Epoch 7/10
50000/50000 [==============================] - 10s 207us/step - loss: 1.2585 - acc: 0.5525 - val_loss: 1.

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 16s 321us/step - loss: 1.8088 - acc: 0.3633 - val_loss: 1.7325 - val_acc: 0.4023
Epoch 2/10
50000/50000 [==============================] - 11s 230us/step - loss: 1.5665 - acc: 0.4432 - val_loss: 1.6759 - val_acc: 0.4142
Epoch 3/10
50000/50000 [==============================] - 11s 217us/step - loss: 1.4774 - acc: 0.4735 - val_loss: 1.5750 - val_acc: 0.4293
Epoch 4/10
50000/50000 [==============================] - 10s 204us/step - loss: 1.4092 - acc: 0.4973 - val_loss: 1.6025 - val_acc: 0.4247
Epoch 5/10
50000/50000 [==============================] - 9s 184us/step - loss: 1.3533 - acc: 0.5170 - val_loss: 1.5607 - val_acc: 0.4367
Epoch 6/10
50000/50000 [==============================] - 9s 184us/step - loss: 1.3061 - acc: 0.5354 - val_loss: 1.5226 - val_acc: 0.4599
Epoch 7/10
50000/50000 [==============================] - 9s 185us/step - loss: 1.2654 - acc: 0.5515 - val_loss: 1.696

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 13s 259us/step - loss: 1.8118 - acc: 0.3738 - val_loss: 2.1643 - val_acc: 0.2959
Epoch 2/10
50000/50000 [==============================] - 9s 176us/step - loss: 1.5326 - acc: 0.4582 - val_loss: 2.2690 - val_acc: 0.2975
Epoch 3/10
50000/50000 [==============================] - 9s 176us/step - loss: 1.4291 - acc: 0.4948 - val_loss: 2.5414 - val_acc: 0.2352
Epoch 4/10
50000/50000 [==============================] - 9s 177us/step - loss: 1.3553 - acc: 0.5223 - val_loss: 2.6124 - val_acc: 0.2660
Epoch 5/10
50000/50000 [==============================] - 9s 177us/step - loss: 1.2938 - acc: 0.5432 - val_loss: 2.0971 - val_acc: 0.3081
Epoch 6/10
50000/50000 [==============================] - 9s 178us/step - loss: 1.2424 - acc: 0.5609 - val_loss: 2.4198 - val_acc: 0.2816
Epoch 7/10
50000/50000 [==============================] - 9s 180us/step - loss: 1.1997 - acc: 0.5772 - val_loss: 2.1907 -

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 13s 270us/step - loss: 1.8110 - acc: 0.3744 - val_loss: 2.4728 - val_acc: 0.3025
Epoch 2/10
50000/50000 [==============================] - 9s 177us/step - loss: 1.5331 - acc: 0.4585 - val_loss: 2.0188 - val_acc: 0.3220
Epoch 3/10
50000/50000 [==============================] - 9s 178us/step - loss: 1.4261 - acc: 0.4956 - val_loss: 2.2186 - val_acc: 0.3090
Epoch 4/10
50000/50000 [==============================] - 9s 178us/step - loss: 1.3536 - acc: 0.5240 - val_loss: 2.3893 - val_acc: 0.2849
Epoch 5/10
50000/50000 [==============================] - 9s 179us/step - loss: 1.2902 - acc: 0.5441 - val_loss: 2.1568 - val_acc: 0.3223
Epoch 6/10
50000/50000 [==============================] - 9s 178us/step - loss: 1.2468 - acc: 0.5593 - val_loss: 2.0440 - val_acc: 0.3158
Epoch 7/10
50000/50000 [==============================] - 9s 179us/step - loss: 1.1935 - acc: 0.5792 - val_loss: 2.1244 -

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 17s 335us/step - loss: 1.7849 - acc: 0.3822 - val_loss: 1.9959 - val_acc: 0.3334
Epoch 2/10
50000/50000 [==============================] - 9s 178us/step - loss: 1.5155 - acc: 0.4669 - val_loss: 2.0476 - val_acc: 0.3033
Epoch 3/10
50000/50000 [==============================] - 9s 182us/step - loss: 1.4107 - acc: 0.5012 - val_loss: 1.8633 - val_acc: 0.3554
Epoch 4/10
50000/50000 [==============================] - 9s 179us/step - loss: 1.3363 - acc: 0.5304 - val_loss: 2.1986 - val_acc: 0.3109
Epoch 5/10
50000/50000 [==============================] - 9s 180us/step - loss: 1.2775 - acc: 0.5496 - val_loss: 2.0026 - val_acc: 0.3538
Epoch 6/10
50000/50000 [==============================] - 9s 181us/step - loss: 1.2234 - acc: 0.5703 - val_loss: 1.9320 - val_acc: 0.3423
Epoch 7/10
50000/50000 [==============================] - 9s 179us/step - loss: 1.1758 - acc: 0.5870 - val_loss: 2.1369 -

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()